In [1]:
import torch
import imageio
from PINN import PINN
from Net import Net
import numpy as np
import pandas as pd
import os

In [2]:
data_dir = f"{os.getcwd()}/pinn_test_data"
# load in data
df_wind_ch4 = pd.read_csv(data_dir + "/wind_ch4.csv")
df_true_emission = pd.read_csv(data_dir + "/selected_controll_release.csv")
source_points = np.load(data_dir + "/source_points.npy") # shape=(n_source, 3)
sensor_points = np.load(data_dir + "/sensor_points.npy") # shape=(n_sensor, 3)
#col_points = np.load(data_dir + "/col_points.npy")  # shape=(n_col, 3)
df_bounds = pd.read_csv(data_dir + "/bounds.csv", dtype='float32')
x_min = df_bounds['x_min'][0]
x_max = df_bounds['x_max'][0]
y_min = df_bounds['y_min'][0]
y_max = df_bounds['y_max'][0]
z_min = df_bounds['z_min'][0]
z_max = df_bounds['z_max'][0]

x_max = 1
y_max = 1
z_max = 1
sensor_points = np.array([[.5,.5,.5]])

ws = df_wind_ch4['wind_speed.m/s'].to_numpy() # shape=(N_t,)
wd = df_wind_ch4['wind_direction'].to_numpy() # shape=(N_t,)
ch4 = np.transpose(df_wind_ch4.iloc[:, 3:].to_numpy()) # shape=(N_obs, N_t)
sensor_names = df_wind_ch4.columns[3:]

In [3]:
model = PINN([50,50,50])
model.set_location(sensor_points,[1,x_max,y_max,z_max],source_values=[1])
tfinal = 1.

In [4]:
optimizer = torch.optim.Adam(model.net.parameters(), lr=1e-3)
from torch.optim.lr_scheduler import ExponentialLR

# scheduler = ExponentialLR(optimizer, gamma=0.999)  # Decay LR by 5% every epoch

In [5]:
n= int(5e3)
sn = 250
best_loss = np.inf
sigma=.025
max_epochs = int(5e2)
print_freq = 1

sampling_freq = 10 # how often to resample collocation and source points

for epoch in range(max_epochs):

    if epoch % sampling_freq == 0:
        source_points,uv_points,source_values = model.source_points(sn,sigma)
        ic_col = torch.cat([torch.rand(sn,1)*tfinal*.1, torch.rand(sn,1)*x_max, torch.rand(sn,1)*y_max, torch.rand(sn,1)*z_max], dim=1)
        collocation_points = torch.cat([torch.rand(n,1)*tfinal, torch.rand(n,1)*x_max, torch.rand(n,1)*y_max, torch.rand(n,1)*z_max], dim=1)
        xxx = torch.tensor(model.source_locs).repeat(len(collocation_points),1)
        ten = torch.sqrt(torch.sum(torch.square(collocation_points[:,1:] - xxx),axis=1))
        collocation_points = collocation_points[ten>sigma*3]
    # collocation_points = torch.cat([collocation_points,ic_col])
    # collocation_points.requires_grad=True

    uv = torch.ones(len(collocation_points),2)*.5
    optimizer.zero_grad()

    loss_1 ,pde_1 = model.loss_function(collocation_points,uv) # PDE residual loss
    loss_2,pde_2 = model.loss_function(source_points, uv_points, source_term = source_values) # source term PDE residual loss 
    loss_3 ,pde_3 = model.loss_function(torch.concat([collocation_points,source_points]),torch.concat([collocation_points,source_points]))

    loss = loss_1 + loss_2
    # loss = loss_2*100
    # loss = loss_1

    # # print loss at first epoch
    # if epoch == 0:
    #     print('epoch: %d, loss: %1.3e, pde_res: %1.3e, source_loss: %1.3e' % (epoch, loss.item(), loss_1.item(), loss_2.item()))

    loss.backward()

    # compute norm of gradient of the network
    grad_norm = 0
    for p in model.net.parameters():
        grad_norm += p.grad.data.norm(2).item()**2
    grad_norm = grad_norm**0.5


    if loss.item() < best_loss:
        torch.save(model,'best_mod.m')
    optimizer.step()
    # scheduler.step()

    if epoch % print_freq == 0:

        # print epoch and loss using %1.3e format
        print('epoch: %d, loss: %1.3e, grad_norm: %1.3e, pde_res: %1.3e, source_loss: %1.3e' % (epoch, loss.item(), grad_norm, loss_1.item(), loss_2.item()))

        # print(epoch, loss.item())
        # print(loss_2.item(),loss_1.item())


epoch: 0, loss: 5.478e+01, grad_norm: 3.368e+01, pde_res: 2.235e-02, source_loss: 5.476e+01
epoch: 1, loss: 5.408e+01, grad_norm: 3.297e+01, pde_res: 3.359e-02, source_loss: 5.405e+01
epoch: 2, loss: 5.339e+01, grad_norm: 3.245e+01, pde_res: 4.942e-02, source_loss: 5.335e+01
epoch: 3, loss: 5.271e+01, grad_norm: 3.200e+01, pde_res: 6.995e-02, source_loss: 5.264e+01
epoch: 4, loss: 5.204e+01, grad_norm: 3.198e+01, pde_res: 9.487e-02, source_loss: 5.195e+01
epoch: 5, loss: 5.141e+01, grad_norm: 3.212e+01, pde_res: 1.243e-01, source_loss: 5.129e+01
epoch: 6, loss: 5.080e+01, grad_norm: 3.223e+01, pde_res: 1.585e-01, source_loss: 5.064e+01
epoch: 7, loss: 5.021e+01, grad_norm: 3.240e+01, pde_res: 1.978e-01, source_loss: 5.001e+01
epoch: 8, loss: 4.959e+01, grad_norm: 3.273e+01, pde_res: 2.424e-01, source_loss: 4.934e+01
epoch: 9, loss: 4.895e+01, grad_norm: 3.309e+01, pde_res: 2.929e-01, source_loss: 4.866e+01
epoch: 10, loss: 4.283e+01, grad_norm: 3.102e+01, pde_res: 3.510e-01, source_los

In [6]:
print(x_max)

1


In [7]:
model = torch.load('best_mod.m')

/var/folders/8t/rfcrq95s3cj0_l1g0lq3_v540000gn/T/ipykernel_14032/3773580750.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('best_mod.m')


In [8]:
import matplotlib.pyplot as plt

# load the best model 
Z_value = .5
# net.load_state_dict(torch.load('best_model.pth'))

# Define the grid and time steps
n= 100
x_grid = np.linspace(0, x_max, n)
y_grid = np.linspace(0, y_max, n)
z_grid = np.linspace(0, z_max, n)

X, Y, = np.meshgrid(x_grid, y_grid)
Z= X * 0 + Z_value

grid_points = np.vstack([X.ravel(), Y.ravel(), Z.ravel()]).T
# grid_points = np.vstack([X.ravel(), Y.ravel(), Z.ravel()]).T  # Flatten the grid
grid_points = torch.tensor(grid_points, dtype=torch.float32)

time_steps = np.linspace(0, tfinal, 20)  # 10 time steps from 0 to 1


'''
# Plot the concentration over time
for t in time_steps:
    t_tensor = torch.full((grid_points.shape[0], 1), t, dtype=torch.float32)  # Time input
    concentration = net(grid_points, t_tensor).cpu().detach().numpy().reshape(100, 100)  # Predict and reshape
    
    plt.figure()
    plt.contourf(X, Y, concentration, levels=50, cmap='viridis', vmin=0, vmax=10)  # Plot concentration as a contour plot
    plt.plot(source_loc[0,0].cpu(), source_loc[0,1].cpu(), 'ro', label='Source Location')  # Plot the source location
    plt.colorbar(label='Concentration')
    # fix colorbar from 0 to 1 
    # plt.clim(0, 10.0)  # Set colorbar limits
    plt.title(f"Gas Concentration at t = {t:.2f}")
    plt.xlabel('x')
    plt.ylabel('y')
    # fix colorbar 
    # plt.savefig(f"concentration_t_{t:.2f}.png")  # Save the plot as an image
'''

'\n# Plot the concentration over time\nfor t in time_steps:\n    t_tensor = torch.full((grid_points.shape[0], 1), t, dtype=torch.float32)  # Time input\n    concentration = net(grid_points, t_tensor).cpu().detach().numpy().reshape(100, 100)  # Predict and reshape\n    \n    plt.figure()\n    plt.contourf(X, Y, concentration, levels=50, cmap=\'viridis\', vmin=0, vmax=10)  # Plot concentration as a contour plot\n    plt.plot(source_loc[0,0].cpu(), source_loc[0,1].cpu(), \'ro\', label=\'Source Location\')  # Plot the source location\n    plt.colorbar(label=\'Concentration\')\n    # fix colorbar from 0 to 1 \n    # plt.clim(0, 10.0)  # Set colorbar limits\n    plt.title(f"Gas Concentration at t = {t:.2f}")\n    plt.xlabel(\'x\')\n    plt.ylabel(\'y\')\n    # fix colorbar \n    # plt.savefig(f"concentration_t_{t:.2f}.png")  # Save the plot as an image\n'

In [9]:
# save as a GIF
import imageio
import os

images = []
# source_loc = torch.tensor([[.5,.5,.5]])
source_loc = model.source_locs
for t in time_steps:
    t_tensor = torch.full((grid_points.shape[0], 1), t, dtype=torch.float32)  # Time input
    concentration = model.forward(torch.cat([t_tensor,grid_points],dim=1),scaled=True).cpu().detach().numpy().reshape(100, 100)  # Predict and reshape
    
    plt.figure()
    plt.contourf(X, Y, concentration, levels=50, cmap='viridis', vmin=0, vmax=10.0)  # Plot concentration as a contour plot
    plt.plot(model.source_locs[0,0], model.source_locs[0,1], 'ro', label='Source Location')  # Plot the source location
    plt.colorbar(label='Concentration')
    plt.title(f"Gas Concentration at t = {t:.2f}")
    plt.xlabel('x')
    plt.ylabel('y')
    plt.savefig(f"concentration_t_{t:.2f}.png")  # Save the plot as an image
    plt.close()
    
    images.append(imageio.imread(f"concentration_t_{t:.2f}.png"))  # Append the image to the list
    os.remove(f"concentration_t_{t:.2f}.png")  # Remove the image file

imageio.mimsave(f'test_visual.gif', images)  # Save the images as a GIF



/var/folders/8t/rfcrq95s3cj0_l1g0lq3_v540000gn/T/ipykernel_14032/4133055751.py:22: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(f"concentration_t_{t:.2f}.png"))  # Append the image to the list


In [10]:
print(len(model.net.hidden))

4


In [11]:
print(source_points)

tensor([[5.0693e-01, 5.4631e-01, 4.4539e-01, 5.0281e-01],
        [6.4213e-01, 5.3763e-01, 4.5460e-01, 4.8149e-01],
        [1.3002e-01, 4.8154e-01, 4.6668e-01, 5.4990e-01],
        [5.2779e-01, 5.2010e-01, 5.1195e-01, 4.7588e-01],
        [9.9788e-01, 4.9930e-01, 5.2245e-01, 4.9994e-01],
        [3.9382e-01, 4.7434e-01, 5.0361e-01, 4.5800e-01],
        [1.0139e-02, 4.7225e-01, 5.0037e-01, 5.0518e-01],
        [2.8969e-01, 5.0483e-01, 4.7941e-01, 5.2670e-01],
        [8.0901e-01, 5.2211e-01, 5.3512e-01, 5.1937e-01],
        [6.6565e-01, 4.5441e-01, 5.1447e-01, 4.8538e-01],
        [8.6781e-01, 5.3913e-01, 4.8684e-01, 4.3250e-01],
        [3.5556e-01, 5.3125e-01, 4.7830e-01, 4.7948e-01],
        [8.8042e-01, 4.8033e-01, 4.8758e-01, 5.0261e-01],
        [5.4470e-01, 5.2459e-01, 4.8026e-01, 4.9294e-01],
        [1.8844e-01, 5.2822e-01, 4.9697e-01, 5.1804e-01],
        [3.6648e-01, 5.4381e-01, 4.8775e-01, 5.0425e-01],
        [1.1713e-01, 5.0692e-01, 5.1384e-01, 4.9431e-01],
        [1.599

In [12]:
print(source_values)

tensor([[0.0164],
        [0.0471],
        [0.0427],
        [0.4053],
        [0.6679],
        [0.1425],
        [0.5286],
        [0.3953],
        [0.1868],
        [0.1352],
        [0.0067],
        [0.2243],
        [0.6450],
        [0.4337],
        [0.4047],
        [0.1883],
        [0.8046],
        [0.1904],
        [0.3575],
        [0.7259],
        [0.1122],
        [0.3577],
        [0.8048],
        [0.1768],
        [0.4877],
        [0.7144],
        [0.6384],
        [0.2763],
        [0.0634],
        [0.0780],
        [0.2012],
        [0.9084],
        [0.0091],
        [0.1757],
        [0.1788],
        [0.1163],
        [0.1632],
        [0.4302],
        [0.2704],
        [0.1496],
        [0.0367],
        [0.1516],
        [0.1572],
        [0.4047],
        [0.5987],
        [0.1731],
        [0.0094],
        [0.0316],
        [0.1721],
        [0.0793],
        [0.7676],
        [0.3864],
        [0.6857],
        [0.5525],
        [0.0894],
        [0